# Advanced Model Integration Tutorial

This tutorial covers advanced techniques for integrating multiple AI models and providers with the Orchestrator framework.

## Overview

The Orchestrator framework supports:

- **Multiple Providers**: OpenAI, Anthropic, Google, local models
- **Dynamic Model Selection**: Automatic best-fit model selection
- **Model Capabilities**: Understanding what each model can do
- **Fallback Strategies**: Automatic failover between models
- **Cost Optimization**: Choosing models based on cost and performance
- **Streaming Support**: Real-time response streaming

## Setup

Let's start by importing the necessary components:

In [ ]:
import sys
sys.path.insert(0, '../src')

import json
from typing import Dict, List, Any

from orchestrator.core.model import Model, MockModel, ModelCapabilities
from orchestrator.models.model_registry import ModelRegistry
from orchestrator.core.task import Task
from orchestrator.core.pipeline import Pipeline
from orchestrator.orchestrator import Orchestrator
from orchestrator.state.state_manager import InMemoryStateManager

print("✅ Advanced model integration components imported!")

## Model Capabilities and Requirements

Understanding model capabilities is crucial for effective model selection:

In [ ]:
# Define capabilities for different types of models

# High-end creative model
creative_capabilities = ModelCapabilities(
    supported_tasks=["generate", "creative_writing", "storytelling"],
    max_tokens=4096,
    supports_streaming=True,
    supports_structured_output=False,
    languages=["en", "es", "fr", "de"],
    response_formats=["text", "markdown"],
    cost_per_token=0.002,
    latency_ms=1500
)

# Analytical model
analytical_capabilities = ModelCapabilities(
    supported_tasks=["analyze", "classify", "extract", "summarize"],
    max_tokens=2048,
    supports_streaming=False,
    supports_structured_output=True,
    languages=["en"],
    response_formats=["text", "json", "structured"],
    cost_per_token=0.001,
    latency_ms=800
)

# Fast, efficient model
efficient_capabilities = ModelCapabilities(
    supported_tasks=["generate", "translate", "simple_qa"],
    max_tokens=1024,
    supports_streaming=True,
    supports_structured_output=False,
    languages=["en", "es", "fr", "de", "it", "pt", "ja", "ko", "zh"],
    response_formats=["text"],
    cost_per_token=0.0005,
    latency_ms=400
)

print("🎭 Creative Model Capabilities:")
print(f"   Tasks: {creative_capabilities.supported_tasks}")
print(f"   Max tokens: {creative_capabilities.max_tokens}")
print(f"   Cost per token: ${creative_capabilities.cost_per_token}")

print("\n🔍 Analytical Model Capabilities:")
print(f"   Tasks: {analytical_capabilities.supported_tasks}")
print(f"   Structured output: {analytical_capabilities.supports_structured_output}")
print(f"   Languages: {len(analytical_capabilities.languages)}")

print("\n⚡ Efficient Model Capabilities:")
print(f"   Tasks: {efficient_capabilities.supported_tasks}")
print(f"   Languages: {len(efficient_capabilities.languages)}")
print(f"   Latency: {efficient_capabilities.latency_ms}ms")

## Creating a Model Registry

The model registry helps manage multiple models and their selection:

In [ ]:
# Create mock models with different capabilities
creative_model = MockModel(
    name="creative-gpt",
    provider="openai",
    capabilities=creative_capabilities
)

analytical_model = MockModel(
    name="claude-analytical", 
    provider="anthropic",
    capabilities=analytical_capabilities
)

efficient_model = MockModel(
    name="gemini-efficient",
    provider="google",
    capabilities=efficient_capabilities
)

# Create model registry and register models
registry = ModelRegistry()
registry.register_model(creative_model)
registry.register_model(analytical_model)
registry.register_model(efficient_model)

print(f"🗂️ Model Registry created with {len(registry.list_models())} models:")
for model_name in registry.list_models():
    model = registry.get_model(model_name)
    print(f"   📱 {model.name} ({model.provider}) - {len(model.capabilities.supported_tasks)} tasks")

## Intelligent Model Selection

The registry can automatically select the best model based on requirements:

In [ ]:
# Test different selection criteria

# 1. Select by task type
creative_requirements = {"tasks": ["creative_writing"]}
selected = registry.select_model(creative_requirements)
print(f"🎨 Best model for creative writing: {selected.name if selected else 'None'}")

# 2. Select by structured output capability
structured_requirements = {"supports_structured_output": True}
selected = registry.select_model(structured_requirements)
print(f"📊 Best model for structured output: {selected.name if selected else 'None'}")

# 3. Select by cost optimization
cost_requirements = {"max_cost_per_token": 0.001}
selected = registry.select_model(cost_requirements)
print(f"💰 Most cost-effective model: {selected.name if selected else 'None'}")

# 4. Select by language support
multilingual_requirements = {"languages": ["zh", "ja"]}
selected = registry.select_model(multilingual_requirements)
print(f"🌏 Best model for Chinese/Japanese: {selected.name if selected else 'None'}")

# 5. Select by performance requirements
performance_requirements = {"max_latency_ms": 500}
selected = registry.select_model(performance_requirements)
print(f"⚡ Fastest model: {selected.name if selected else 'None'}")

## Advanced Model Selection Strategies

Let's implement some advanced selection strategies:

In [ ]:
class AdvancedModelSelector:
    """Advanced model selection with custom strategies."""
    
    def __init__(self, registry: ModelRegistry):
        self.registry = registry
    
    def select_by_cost_performance_ratio(self, task_type: str) -> Model:
        """Select model with best cost/performance ratio for a task."""
        suitable_models = []
        
        for model_name in self.registry.list_models():
            model = self.registry.get_model(model_name)
            if task_type in model.capabilities.supported_tasks:
                # Calculate cost-performance score (lower is better)
                cost_score = model.capabilities.cost_per_token or 0.01
                latency_score = (model.capabilities.latency_ms or 1000) / 1000
                combined_score = cost_score * latency_score
                
                suitable_models.append((model, combined_score))
        
        if suitable_models:
            # Return model with lowest cost-performance score
            return min(suitable_models, key=lambda x: x[1])[0]
        return None
    
    def select_with_fallback(self, requirements: Dict[str, Any]) -> List[Model]:
        """Select primary model with fallback options."""
        models = []
        
        # Primary: exact match
        primary = self.registry.select_model(requirements)
        if primary:
            models.append(primary)
        
        # Fallback 1: relax some requirements
        relaxed_requirements = requirements.copy()
        if "max_latency_ms" in relaxed_requirements:
            relaxed_requirements["max_latency_ms"] *= 2
        
        fallback1 = self.registry.select_model(relaxed_requirements)
        if fallback1 and fallback1 not in models:
            models.append(fallback1)
        
        # Fallback 2: any model supporting the task
        if "tasks" in requirements:
            task_requirements = {"tasks": requirements["tasks"]}
            fallback2 = self.registry.select_model(task_requirements)
            if fallback2 and fallback2 not in models:
                models.append(fallback2)
        
        return models
    
    def select_by_load_balancing(self, task_type: str) -> Model:
        """Select model based on current load (simulated)."""
        suitable_models = []
        
        for model_name in self.registry.list_models():
            model = self.registry.get_model(model_name)
            if task_type in model.capabilities.supported_tasks:
                # Simulate current load (in real implementation, this would be actual metrics)
                import random
                simulated_load = random.uniform(0.1, 0.9)
                suitable_models.append((model, simulated_load))
        
        if suitable_models:
            # Return model with lowest simulated load
            return min(suitable_models, key=lambda x: x[1])[0]
        return None

# Test advanced selection strategies
selector = AdvancedModelSelector(registry)

# Cost-performance optimization
best_ratio_model = selector.select_by_cost_performance_ratio("generate")
print(f"💡 Best cost/performance for generation: {best_ratio_model.name if best_ratio_model else 'None'}")

# Fallback strategy
strict_requirements = {
    "tasks": ["analyze"],
    "supports_structured_output": True,
    "max_latency_ms": 500
}
fallback_models = selector.select_with_fallback(strict_requirements)
print(f"🔄 Fallback chain: {[m.name for m in fallback_models]}")

# Load balancing
load_balanced_model = selector.select_by_load_balancing("generate")
print(f"⚖️ Load balanced selection: {load_balanced_model.name if load_balanced_model else 'None'}")

## Model-Aware Pipeline Execution

Create a pipeline that intelligently assigns tasks to appropriate models:

In [ ]:
# Create a pipeline with model-specific tasks
smart_pipeline = Pipeline(
    id="smart_model_pipeline",
    name="Smart Model Selection Pipeline",
    description="Demonstrates intelligent model assignment"
)

# Task 1: Creative content generation (should use creative model)
creative_task = Task(
    id="create_story",
    name="Create Story",
    action="creative_writing",
    parameters={
        "prompt": "Write a short science fiction story about time travel",
        "style": "dramatic",
        "length": "medium"
    },
    metadata={
        "preferred_model_type": "creative",
        "requirements": {
            "tasks": ["creative_writing"],
            "min_tokens": 2000
        }
    }
)

# Task 2: Analysis (should use analytical model)
analysis_task = Task(
    id="analyze_story",
    name="Analyze Story",
    action="analyze",
    parameters={
        "prompt": "Analyze the narrative structure and themes in: {create_story.result}",
        "output_format": "structured"
    },
    dependencies=["create_story"],
    metadata={
        "preferred_model_type": "analytical",
        "requirements": {
            "tasks": ["analyze"],
            "supports_structured_output": True
        }
    }
)

# Task 3: Quick summary (should use efficient model)
summary_task = Task(
    id="summarize",
    name="Quick Summary",
    action="generate",
    parameters={
        "prompt": "Create a brief 50-word summary of: {create_story.result}",
        "max_length": 50
    },
    dependencies=["create_story"],
    metadata={
        "preferred_model_type": "efficient",
        "requirements": {
            "tasks": ["generate"],
            "max_latency_ms": 600,
            "max_cost_per_token": 0.001
        }
    }
)

# Add tasks to pipeline
smart_pipeline.add_task(creative_task)
smart_pipeline.add_task(analysis_task)
smart_pipeline.add_task(summary_task)

print(f"🧠 Smart pipeline created with {len(smart_pipeline)} tasks")
print(f"📋 Execution order: {smart_pipeline.get_execution_order()}")

# Show model assignments
print("\n🎯 Recommended model assignments:")
for task_id in smart_pipeline:
    task = smart_pipeline.get_task(task_id)
    requirements = task.metadata.get("requirements", {})
    recommended_models = selector.select_with_fallback(requirements)
    
    print(f"   {task.name}:")
    if recommended_models:
        primary = recommended_models[0]
        print(f"     Primary: {primary.name} ({primary.provider})")
        if len(recommended_models) > 1:
            fallbacks = [f"{m.name} ({m.provider})" for m in recommended_models[1:]]
            print(f"     Fallbacks: {', '.join(fallbacks)}")
    else:
        print("     No suitable model found")

## Setting Up Mock Responses

Configure our mock models with appropriate responses:

In [ ]:
# Configure creative model responses
creative_story = """
Dr. Sarah Chen stared at the temporal displacement device, her life's work finally complete. 
The swirling vortex of blue energy hummed with possibilities. She had spent fifteen years 
perfecting the equations, but nothing could prepare her for what happened next.

As she stepped through the portal, time fractured around her. She emerged in 1955, in her 
grandfather's laboratory. But something was wrong—the equations on his blackboard were 
identical to hers. With growing horror, she realized the truth: she hadn't invented time 
travel. She had inherited it.

The device in 1955 activated, sending her grandfather forward to 2024. In the present, 
an old man she'd never met knocked on her door, claiming to be her grandfather, 
carrying blueprints that looked exactly like her own.

Time, it seemed, was a closed loop—and she was both its beginning and its end.
""".strip()

creative_model.set_response(
    "Write a short science fiction story about time travel",
    creative_story
)

# Configure analytical model responses
analysis_result = {
    "narrative_structure": {
        "type": "circular_narrative",
        "elements": ["setup", "discovery", "revelation", "closed_loop"]
    },
    "themes": [
        "causality_paradox",
        "scientific_inheritance", 
        "temporal_determinism",
        "family_legacy"
    ],
    "literary_devices": [
        "foreshadowing",
        "dramatic_irony",
        "temporal_symmetry"
    ],
    "tone": "mysterious_and_contemplative",
    "character_development": "revelation_driven"
}

analytical_model.set_response(
    "Analyze the narrative structure and themes",
    json.dumps(analysis_result, indent=2)
)

# Configure efficient model responses
efficient_model.set_response(
    "Create a brief 50-word summary",
    "Dr. Sarah Chen discovers her time travel device was inherited from her grandfather "
    "through a temporal paradox. Her invention creates a closed loop where past and "
    "future are connected, revealing that time travel was both her discovery and her legacy."
)

print("✅ Mock responses configured for all models")
print(f"   📝 Creative story: {len(creative_story)} characters")
print(f"   📊 Analysis structure: {len(analysis_result)} sections")
print("   📄 Summary: concise overview")

## Smart Orchestrator with Model Assignment

Create an orchestrator that automatically assigns tasks to optimal models:

In [ ]:
class SmartOrchestrator(Orchestrator):
    """Enhanced orchestrator with intelligent model assignment."""
    
    def __init__(self, state_manager, model_registry: ModelRegistry):
        super().__init__(state_manager)
        self.model_registry = model_registry
        self.selector = AdvancedModelSelector(model_registry)
        
        # Register all models from registry
        for model_name in model_registry.list_models():
            model = model_registry.get_model(model_name)
            self.register_model(model)
    
    def assign_optimal_models(self, pipeline: Pipeline) -> Dict[str, str]:
        """Assign optimal models to pipeline tasks."""
        assignments = {}
        
        for task_id in pipeline:
            task = pipeline.get_task(task_id)
            requirements = task.metadata.get("requirements", {})
            
            # Try to find optimal model
            fallback_models = self.selector.select_with_fallback(requirements)
            
            if fallback_models:
                assignments[task_id] = fallback_models[0].name
                print(f"✅ Assigned {task.name} → {fallback_models[0].name}")
                
                # Store fallback options in task metadata
                if len(fallback_models) > 1:
                    fallback_names = [m.name for m in fallback_models[1:]]
                    task.metadata["fallback_models"] = fallback_names
                    print(f"   Fallbacks: {', '.join(fallback_names)}")
            else:
                print(f"⚠️ No suitable model found for {task.name}")
        
        return assignments
    
    async def execute_with_smart_assignment(self, pipeline: Pipeline) -> bool:
        """Execute pipeline with automatic model assignment."""
        print(f"🧠 Executing smart pipeline: {pipeline.name}")
        
        # Assign optimal models
        assignments = self.assign_optimal_models(pipeline)
        
        # Execute with automatic failover
        for task_id in pipeline:
            task = pipeline.get_task(task_id)
            
            if task_id in assignments:
                primary_model = assignments[task_id]
                fallbacks = task.metadata.get("fallback_models", [])
                
                # Try primary model first
                success = await self._execute_task_with_model(task, primary_model)
                
                # Try fallbacks if primary fails
                if not success:
                    for fallback_model in fallbacks:
                        print(f"🔄 Trying fallback model: {fallback_model}")
                        success = await self._execute_task_with_model(task, fallback_model)
                        if success:
                            break
                
                if not success:
                    print(f"❌ All models failed for task: {task.name}")
                    return False
        
        return True
    
    async def _execute_task_with_model(self, task: Task, model_name: str) -> bool:
        """Execute a single task with specified model."""
        try:
            model = self.models[model_name]
            
            # Simulate task execution
            task.start()
            
            # Extract prompt from parameters
            prompt = task.parameters.get("prompt", "")
            
            # Execute based on action type
            if task.action in ["generate", "creative_writing"]:
                result = await model.generate(prompt, **task.parameters)
            elif task.action == "analyze":
                # For structured output, try generate_structured if available
                if (hasattr(model, 'generate_structured') and 
                    model.capabilities.supports_structured_output):
                    schema = {"type": "object"}  # Basic schema
                    result = await model.generate_structured(prompt, schema)
                else:
                    result = await model.generate(prompt, **task.parameters)
            else:
                result = await model.generate(prompt, **task.parameters)
            
            task.complete(result)
            print(f"✅ Completed {task.name} with {model_name}")
            return True
            
        except Exception as e:
            print(f"❌ Failed {task.name} with {model_name}: {e}")
            task.fail(e)
            return False

# Create smart orchestrator
state_manager = InMemoryStateManager()
smart_orchestrator = SmartOrchestrator(state_manager, registry)

print(f"🧠 Smart orchestrator created with {len(smart_orchestrator.models)} models")
print(f"📊 Available models: {list(smart_orchestrator.models.keys())}")

## Executing the Smart Pipeline

Now let's execute our pipeline with intelligent model assignment:

In [ ]:
async def run_smart_pipeline():
    """Execute the smart pipeline with automatic model assignment."""
    print("🚀 Starting smart pipeline execution...\n")
    
    # Show initial pipeline state
    print(f"📊 Pipeline: {smart_pipeline.name}")
    print(f"📋 Tasks: {len(smart_pipeline)}")
    print(f"⚡ Execution order: {smart_pipeline.get_execution_order()}")
    print()
    
    # Execute with smart assignment
    success = await smart_orchestrator.execute_with_smart_assignment(smart_pipeline)
    
    print(f"\n{'✅' if success else '❌'} Pipeline execution {'completed' if success else 'failed'}")
    
    # Show results
    if success:
        print("\n📊 Task Results:")
        print("=" * 50)
        
        for task_id in smart_pipeline:
            task = smart_pipeline.get_task(task_id)
            print(f"\n🔸 {task.name}")
            print(f"   Status: {task.status}")
            print(f"   Action: {task.action}")
            
            if task.result:
                # Format result for display
                result_text = str(task.result)
                if len(result_text) > 300:
                    result_text = result_text[:300] + "..."
                print(f"   Result: {result_text}")
            
            # Show model assignment info
            if "fallback_models" in task.metadata:
                print(f"   Fallbacks available: {len(task.metadata['fallback_models'])}")
    
    return success

# Execute the smart pipeline
result = await run_smart_pipeline()

## Model Performance Monitoring

Track model performance and costs across executions:

In [ ]:
# Simulate performance tracking
class ModelPerformanceTracker:
    """Track model performance metrics."""
    
    def __init__(self):
        self.metrics = {}
    
    def record_execution(self, model_name: str, task_type: str, 
                        success: bool, latency_ms: int, tokens_used: int):
        """Record a model execution."""
        if model_name not in self.metrics:
            self.metrics[model_name] = {
                "total_executions": 0,
                "successful_executions": 0,
                "total_latency_ms": 0,
                "total_tokens": 0,
                "task_types": {}
            }
        
        model_metrics = self.metrics[model_name]
        model_metrics["total_executions"] += 1
        
        if success:
            model_metrics["successful_executions"] += 1
            model_metrics["total_latency_ms"] += latency_ms
            model_metrics["total_tokens"] += tokens_used
        
        # Track by task type
        if task_type not in model_metrics["task_types"]:
            model_metrics["task_types"][task_type] = {"count": 0, "success_rate": 0}
        
        model_metrics["task_types"][task_type]["count"] += 1
        if success:
            task_metrics = model_metrics["task_types"][task_type]
            task_metrics["success_rate"] = (
                (task_metrics["success_rate"] * (task_metrics["count"] - 1) + 1) /
                task_metrics["count"]
            )
    
    def get_performance_report(self) -> Dict[str, Any]:
        """Generate performance report."""
        report = {}
        
        for model_name, metrics in self.metrics.items():
            total_exec = metrics["total_executions"]
            successful_exec = metrics["successful_executions"]
            
            success_rate = successful_exec / total_exec if total_exec > 0 else 0
            avg_latency = (
                metrics["total_latency_ms"] / successful_exec 
                if successful_exec > 0 else 0
            )
            
            report[model_name] = {
                "success_rate": success_rate,
                "average_latency_ms": avg_latency,
                "total_executions": total_exec,
                "total_tokens": metrics["total_tokens"],
                "task_performance": metrics["task_types"]
            }
        
        return report

# Simulate some performance data
tracker = ModelPerformanceTracker()

# Simulate executions for our models
import random

# Creative model performance
for _ in range(10):
    tracker.record_execution(
        "creative-gpt", "creative_writing", 
        True, random.randint(1000, 2000), random.randint(200, 500)
    )

# Analytical model performance  
for _ in range(15):
    tracker.record_execution(
        "claude-analytical", "analyze",
        random.choice([True, True, True, False]),  # 75% success rate
        random.randint(600, 1200), random.randint(100, 300)
    )

# Efficient model performance
for _ in range(20):
    tracker.record_execution(
        "gemini-efficient", "generate",
        True, random.randint(200, 600), random.randint(50, 150)
    )

# Generate performance report
performance_report = tracker.get_performance_report()

print("📊 Model Performance Report:")
print("=" * 50)

for model_name, metrics in performance_report.items():
    print(f"\n🤖 {model_name}:")
    print(f"   Success Rate: {metrics['success_rate']:.1%}")
    print(f"   Avg Latency: {metrics['average_latency_ms']:.0f}ms")
    print(f"   Total Executions: {metrics['total_executions']}")
    print(f"   Total Tokens: {metrics['total_tokens']:,}")
    
    # Show task-specific performance
    if metrics['task_performance']:
        print("   Task Performance:")
        for task_type, task_metrics in metrics['task_performance'].items():
            print(f"     {task_type}: {task_metrics['success_rate']:.1%} ({task_metrics['count']} executions)")

## Cost Analysis and Optimization

Analyze costs across different models and execution strategies:

In [ ]:
class CostAnalyzer:
    """Analyze and optimize costs across models."""
    
    def __init__(self, registry: ModelRegistry, tracker: ModelPerformanceTracker):
        self.registry = registry
        self.tracker = tracker
    
    def calculate_execution_costs(self) -> Dict[str, float]:
        """Calculate total costs per model."""
        costs = {}
        
        for model_name in self.registry.list_models():
            model = self.registry.get_model(model_name)
            cost_per_token = model.capabilities.cost_per_token or 0
            
            # Get token usage from performance tracker
            performance = self.tracker.get_performance_report()
            if model_name in performance:
                total_tokens = performance[model_name]["total_tokens"]
                total_cost = total_tokens * cost_per_token
                costs[model_name] = total_cost
            else:
                costs[model_name] = 0.0
        
        return costs
    
    def analyze_cost_efficiency(self) -> Dict[str, Dict[str, float]]:
        """Analyze cost efficiency (cost per successful execution)."""
        efficiency = {}
        costs = self.calculate_execution_costs()
        performance = self.tracker.get_performance_report()
        
        for model_name, total_cost in costs.items():
            if model_name in performance:
                perf = performance[model_name]
                successful_exec = perf["total_executions"] * perf["success_rate"]
                
                if successful_exec > 0:
                    cost_per_success = total_cost / successful_exec
                    time_per_success = perf["average_latency_ms"] / 1000  # Convert to seconds
                    
                    efficiency[model_name] = {
                        "cost_per_successful_execution": cost_per_success,
                        "time_per_successful_execution": time_per_success,
                        "cost_efficiency_score": cost_per_success / max(time_per_success, 0.1)
                    }
        
        return efficiency
    
    def recommend_cost_optimization(self) -> List[str]:
        """Provide cost optimization recommendations."""
        recommendations = []
        efficiency = self.analyze_cost_efficiency()
        
        if not efficiency:
            return ["No performance data available for analysis"]
        
        # Find most cost-efficient model
        most_efficient = min(efficiency.items(), 
                           key=lambda x: x[1]["cost_per_successful_execution"])
        
        recommendations.append(
            f"Most cost-efficient model: {most_efficient[0]} "
            f"(${most_efficient[1]['cost_per_successful_execution']:.4f} per success)"
        )
        
        # Find fastest model
        fastest = min(efficiency.items(),
                     key=lambda x: x[1]["time_per_successful_execution"])
        
        recommendations.append(
            f"Fastest model: {fastest[0]} "
            f"({fastest[1]['time_per_successful_execution']:.2f}s per success)"
        )
        
        # Best overall efficiency (cost/time ratio)
        best_overall = min(efficiency.items(),
                          key=lambda x: x[1]["cost_efficiency_score"])
        
        recommendations.append(
            f"Best cost/time ratio: {best_overall[0]} "
            f"(efficiency score: {best_overall[1]['cost_efficiency_score']:.4f})"
        )
        
        return recommendations

# Perform cost analysis
cost_analyzer = CostAnalyzer(registry, tracker)

# Calculate costs
execution_costs = cost_analyzer.calculate_execution_costs()
cost_efficiency = cost_analyzer.analyze_cost_efficiency()
recommendations = cost_analyzer.recommend_cost_optimization()

print("💰 Cost Analysis Report:")
print("=" * 50)

print("\n📊 Total Execution Costs:")
for model_name, cost in execution_costs.items():
    print(f"   {model_name}: ${cost:.4f}")

print("\n⚡ Cost Efficiency Analysis:")
for model_name, metrics in cost_efficiency.items():
    print(f"   {model_name}:")
    print(f"     Cost per success: ${metrics['cost_per_successful_execution']:.4f}")
    print(f"     Time per success: {metrics['time_per_successful_execution']:.2f}s")
    print(f"     Efficiency score: {metrics['cost_efficiency_score']:.4f}")

print("\n🎯 Optimization Recommendations:")
for i, recommendation in enumerate(recommendations, 1):
    print(f"   {i}. {recommendation}")

## Summary

In this tutorial, you learned:

1. **Model Capabilities**: Defining and understanding what each model can do
2. **Model Registry**: Managing multiple models and their selection
3. **Intelligent Selection**: Automatic model assignment based on requirements
4. **Fallback Strategies**: Handling model failures gracefully
5. **Performance Monitoring**: Tracking model execution metrics
6. **Cost Optimization**: Analyzing and optimizing model usage costs

## Key Benefits

- **🎯 Optimal Performance**: Right model for each task
- **💰 Cost Control**: Automatic cost optimization
- **🔄 Reliability**: Fallback mechanisms prevent failures
- **📊 Insights**: Performance monitoring and analytics
- **🚀 Scalability**: Easy addition of new models and providers
- **🧠 Intelligence**: Automatic decision making

## Best Practices

1. **Define Clear Capabilities**: Accurately specify what each model can do
2. **Set Up Fallbacks**: Always have backup models for critical tasks
3. **Monitor Performance**: Track success rates, latency, and costs
4. **Optimize Regularly**: Use performance data to improve selections
5. **Test Thoroughly**: Validate model assignments before production
6. **Document Requirements**: Clear task requirements enable better selection

## Next Steps

- Explore **Real Provider Integration** (OpenAI, Anthropic, Google)
- Learn about **Custom Model Adapters** for proprietary models
- Try **Advanced Orchestration Patterns** for complex workflows
- Check out **Production Monitoring** and observability

---

**Smart orchestrating! 🧠🎵**